# Routes Optimization from Clermont Warehouse

In [1]:
import pandas as pd
import time

In [2]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [3]:
factors = pd.read_csv('factors.csv')
routes = pd.read_csv('routes_v2.csv')
cities = pd.read_csv('cities.csv')
orders = pd.read_csv('orders.csv')
pack = pd.read_csv('packages.csv')
price = pd.read_csv('pricing.csv')
trucks = pd.read_csv('trucks.csv')
wareh = pd.read_csv('warehouses.csv')

In [4]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calcule la distance en kilomètres entre deux points (lon1, lat1) et (lon2, lat2) 
    sur une sphère (la Terre) en utilisant la formule de Haversine.
    """
    # Conversion des latitudes et longitudes en radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Calcul des différences de latitude et de longitude
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Calcul de la distance en kilomètres
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    km = 6371 * c
    return km

# Initialisation du dataframe 'distance' avec les noms de villes en index et en colonnes
disti = pd.DataFrame(index=cities['city'], columns=cities['city'])

# Calcul de la distance en kilomètres entre chaque paire de villes
for i in cities.index:
    for j in cities.index:
        disti.loc[cities.loc[i,'city'], cities.loc[j,'city']] = haversine(cities.loc[i,'lng'], cities.loc[i,'lat'], cities.loc[j,'lng'], cities.loc[j,'lat'])

# Conversion des distances en nombres décimaux arrondis à 2 décimales
disti = disti.astype(float).round(6)
disti

city,Paris,Nice,Toulouse,Marseille,Rennes,Grenoble,Nantes,Montpellier,Lyon,Rouen,...,Niort,Sarcelles,Pantin,Lorient,Le Blanc-Mesnil,Beauvais,Maisons-Alfort,Hyères,Épinay-sur-Seine,Meaux
city,,,,,,,,,,,,,,,,,,,,,
Paris,0.000000,685.932701,588.157822,660.494275,308.328951,482.786159,342.702025,594.827948,392.047741,112.669040,...,351.637357,15.596691,5.734735,439.934995,12.126891,66.475147,8.436038,701.319641,11.416059,40.842015
Nice,685.932701,0.000000,468.448463,159.502510,847.433143,204.133979,790.410398,272.813797,298.349843,793.316192,...,673.356422,697.442290,687.505298,938.435488,688.901861,749.113323,677.709069,112.295156,696.722073,674.021104
Toulouse,588.157822,468.448463,0.000000,318.738626,556.669655,381.950351,464.863066,195.899592,359.849115,649.756396,...,337.637018,603.753847,593.030451,592.866625,598.252884,649.705773,583.378025,382.712483,598.674075,605.760183
Marseille,660.494275,159.502510,318.738626,0.000000,765.358589,210.382773,695.079110,125.498568,277.140982,757.981418,...,569.858194,674.123058,663.306103,840.465671,666.113807,726.691658,652.939839,64.769924,671.853507,658.095958
Rennes,308.328951,847.433143,556.669655,765.358589,0.000000,652.570095,100.147058,659.805278,559.699041,250.862716,...,219.173347,314.426918,312.935264,131.607318,318.353491,312.863401,313.123878,822.352762,308.149110,349.133825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Beauvais,66.475147,749.113323,649.705773,726.691658,312.863401,546.714969,364.963437,661.134077,457.003665,72.798887,...,394.276357,52.600119,63.391145,442.454002,60.798387,0.000000,73.781662,766.919127,55.059244,77.796818
Maisons-Alfort,8.436038,677.709069,583.378025,652.939839,313.123878,474.635091,345.311605,588.029923,384.056777,121.059455,...,351.134506,21.512131,10.436289,444.695886,14.878265,73.781662,0.000000,693.506337,19.099178,37.151852
Hyères,701.319641,112.295156,382.712483,64.769924,822.352762,230.451203,754.762312,190.265890,310.947351,802.373811,...,630.799656,714.353224,703.753626,900.945363,706.126675,766.919127,693.506337,0.000000,712.564661,695.831497


# Préparation Produit final 

In [5]:
def arrets_to_matrix(arrets):
    liste_villes = []  #les villes
    for liste in arrets:
        for ville in liste:
            if ville not in liste_villes:
                liste_villes.append(ville)
    mini_disti = pd.DataFrame(index=liste_villes, columns=liste_villes)  #dataframe distances intervilles
    for ville_i in liste_villes:
        for ville_j in liste_villes:
            mini_disti.at[ville_i, ville_j] = disti.at[ville_i, ville_j]
    mini_disti_2 = mini_disti
    return mini_disti_2, mini_disti.to_numpy().tolist()

In [6]:
def create_data_model(matrix, num_camion):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = matrix
    data['num_vehicles'] = num_camion
    data['depot'] = 0
    return data

In [7]:
def supprimer_zero(liste):
    return [sous_liste for sous_liste in liste if any(elem != 0 for elem in sous_liste)]

In [8]:
def nb_to_city(liste, df):
    L=[]
    for k in liste:
        l=[]
        for i in k:
            l.append(df.columns[i])
        L.append(l)
    return L

In [9]:
def total_dist(trajets):
    n=0
    for k in trajets:
        for i in range(len(k)-1):
            n += disti[k[i]][k[i+1]]
    return n

# Produit final Clermont

In [10]:
routes_clermont = routes[routes.from_warehouse == 'Clermont-Ferrand']

In [11]:
liste_dates_clermont = routes_clermont['route_date'].tolist()

In [12]:
tot_distance_clermont_preopti = routes_clermont['total_distance'].sum()
tot_distance_clermont_preopti

93153.37654905213

In [13]:
debut = time.time()
trajets_clermont=[]

for d in liste_dates_clermont:
    smol_df = routes_clermont[routes_clermont.route_date == d]
    l=[]
    l = list(smol_df.stops)
    arrets=[]
    for k in l:
        arrets.append(k.split(' > '))
    df_dist, M_dist = arrets_to_matrix(arrets)
    #Début opti
    data = create_data_model(M_dist, 13)
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        900,  # vehicle maximum travel distance => pour respecter la limite de 9h de travail par jour (100km/h)
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    solution = routing.SolveWithParameters(search_parameters)
    if solution:
        print('ok')
    else:
        print('No solution found !')
    routes_opti = []
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route.append(node_index)
            index = solution.Value(routing.NextVar(index))
        route.append(0)  # Ajouter le nœud de départ à la fin de l'itinéraire
        routes_opti.append(route)
    #Fin opti
    routes_opti_num = supprimer_zero(routes_opti)
    routes_opti_villes = nb_to_city(routes_opti_num, df_dist)
    trajets_clermont+=routes_opti_villes  #la liste de tous les trajets

fin = time.time()
temps_execution = (fin - debut)/60
print("Le temps d'exécution du programme est de : {:.2f} min".format(temps_execution))

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
o

In [14]:
trajets_clermont

[['Clermont-Ferrand', 'Bourges', 'Clermont-Ferrand'],
 ['Clermont-Ferrand', 'Villeurbanne', 'Clermont-Ferrand'],
 ['Clermont-Ferrand', 'Villeurbanne', 'Clermont-Ferrand'],
 ['Clermont-Ferrand', 'Bourges', 'Clermont-Ferrand'],
 ['Clermont-Ferrand', 'Saint-Étienne', 'Clermont-Ferrand'],
 ['Clermont-Ferrand', 'Lyon', 'Clermont-Ferrand'],
 ['Clermont-Ferrand',
  'Saint-Étienne',
  'Villeurbanne',
  'Lyon',
  'Clermont-Ferrand'],
 ['Clermont-Ferrand', 'Bourges', 'Clermont-Ferrand'],
 ['Clermont-Ferrand',
  'Saint-Étienne',
  'Villeurbanne',
  'Lyon',
  'Clermont-Ferrand'],
 ['Clermont-Ferrand', 'Bourges', 'Clermont-Ferrand'],
 ['Clermont-Ferrand', 'Lyon', 'Villeurbanne', 'Clermont-Ferrand'],
 ['Clermont-Ferrand',
  'Saint-Étienne',
  'Villeurbanne',
  'Lyon',
  'Clermont-Ferrand'],
 ['Clermont-Ferrand', 'Bourges', 'Clermont-Ferrand'],
 ['Clermont-Ferrand', 'Saint-Étienne', 'Clermont-Ferrand'],
 ['Clermont-Ferrand', 'Lyon', 'Villeurbanne', 'Clermont-Ferrand'],
 ['Clermont-Ferrand', 'Saint-Ét